

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()
cc_apps.info()
cc_apps.describe()
cc_apps.head()

(cc_apps == '?').sum() # Check how many "?" are in the data set
cc_clean = cc_apps.replace("?",np.nan) #Replace with np.nan so pandas can detect missing values
cc_clean.info() #double check if it worked (should see non-null number dropped)
df = cc_clean.copy() #work on a new copy (justincase)
# Inspect unique values in column 0
print(cc_clean[0].unique())

# Or for every column, see the first few uniques:
for col in cc_clean.columns:
    print(col, cc_clean[col].unique()[:10])

for col in df.columns:
    if df[col].dtype =='object':
        mode_val = df[col].value_counts().idxmax()
        df[col].fillna(mode_val,inplace=True)
    else:
        mean_val = df[col].mean()
        df[col].fillna(mean_val,inplace = True)

df_encoded = pd.get_dummies(df, drop_first = True)
x = df_encoded.iloc[:, :-1].values
y = df_encoded.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(
    x, y,
    test_size = 0.3,
    random_state = 42
    )
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#train and evaluate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix #import the needed 
model = LogisticRegression() #start with default
model.fit(X_train_scaled, y_train) #adjust the weighs to minimize errors/learning phase
y_pred = model.predict(X_test_scaled) #examination (testing) phase, after this step, we get y_pred for the next step to compare to the ACTUAL result
confusion_matrix(y_test, y_pred) #test is the 30, not 70 / TP,TN,FP,FN
model.score(X_test_scaled, y_test)

#compare hyperparameters
Cs = [0.01, 0.1, 1, 10, 100] #regularizations strengths to try
penalties = ['l1', 'l2'] # penalty types
solvers = ['liblinear']
param_grid = {
    "C"      : Cs,
    "penalty": penalties,
    "solver" : solvers
}

#setting up and running grid search 
from sklearn.model_selection import GridSearchCV #try all hyperparameter combos + cross-validate
base_model = LogisticRegression(max_iter = 1000) #max_iter to makesure solver have enough steps to solve
grid = GridSearchCV(
    estimator = base_model, #which model to tune
    param_grid = param_grid, #that's the dictionary of hyperparameter choices
    cv = 5, #requests 5-fold cross-validation 
    scoring = "accuracy" # compare models by their average accuracy
)
grid.fit(X_train_scaled, y_train)

#inspecting the best hyperparameters
print("Best parameters: ", grid.best_params_)
print("CV best score: ", grid.best_score_)
print("Test accuracy: ", grid.score(X_test_scaled, y_test))

#grab the best model
best_model = grid.best_estimator_

#make final predictions
y_pred_best = best_model.predict(X_test_scaled)

#compute and display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix (y_test, y_pred_best)
print("Confusion matrix:\n", cm)
best_score = best_model.score(X_test_scaled, y_test)
print("Final test-set accuracy:", round(best_score, 3))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 75.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       678 non-null    object 
 1   1       678 non-nu